code pembelajaran ini dibuat oleh naufal & hepatika...

latihan yang dipelajari meliputi:
    1. load gambar dalam sebuah folder
    2. preprocessing gambar
    3. spliting dataset train test
    4. klasifikasi
    5. model evaluasi

# membaca data dan memberi label

In [1]:
import os
import os.path
import numpy as np
import matplotlib.pyplot as plt
X = []    
import cv2

#menentukan direktori/folder data citra yang akan dibuka
dirname = 'hymenoptera_data/'  

#menentukan ukuran tinggi dan lebar gambar
height = 220
width = 220
dim = (width, height)

#mengumpulkan data citra yang akan dibuka dalam satu array
tampungan= []
for path, subdirs, files in os.walk(dirname):
    for name in files:
        img_path = (os.path.join(path, name))
        if img_path.endswith("jpg"):
            img = cv2.imread(img_path) #baca path data
            #preprocessing data
            convert = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            resized=cv2.resize(convert,dim, interpolation=cv2.INTER_LINEAR)
            blur = cv2.GaussianBlur(resized,(5,5),0)
            
            tampungan.append(blur.flatten())
X = np.array(tampungan)
print(X.shape)

y0 = np.zeros(122)
y1 = np.ones(121)
y2 = np.full((100), 2)
y = []
y = np.concatenate((y0,y1,y2), axis=0)

(343, 145200)


# train test split data

In [3]:
from sklearn.model_selection import GridSearchCV, train_test_split #library untuk train test split

#melakukan splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.20, stratify=y) #train size adalah persentase data test yang di-split

from builtins import range
from builtins import object

num_training = X_train.shape[0]
mask = list(range(num_training))
X_train = X_train[mask]
y_train = y_train[mask]

num_test = X_test.shape[0]
mask = list(range(num_test))
X_test = X_test[mask]
y_test = y_test[mask]

# Mengubah data citra menjadi baris
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))

print("X_train: "+str(X_train.shape))
print("X_test: "+str(X_test.shape))
print("y_train: "+str(y_train.shape))
print("y_test: "+str(y_test.shape))

X_train: (68, 67500)
X_test: (275, 67500)
y_train: (68,)
y_test: (275,)


# uji training data

In [4]:
from sklearn import svm
from sklearn.model_selection import cross_validate, cross_val_score

scoring = {'acc': 'accuracy',
#            'precision': 'precision',
#            'recall': 'recall'
          }

# membangun model SVM
model = svm.SVC(C=1.0, kernel="linear" ,decision_function_shape='ovr', probability=True)
score = cross_validate(model, X_train, y_train, scoring=scoring,
                         cv=5, return_train_score=True)
print(score['test_acc'].mean())

0.5758241758241758


In [5]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer

def classification_report_with_accuracy_score(y_true, y_pred):
    print (classification_report(y_true, y_pred)) # print hasil klasifikasi
    return accuracy_score(y_true, y_pred) # return skor akurasi
    
# Nested CV dengan parameter yang dioptimasi
nested_score = cross_val_score(model, X_train, y_train, cv=5, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score.mean())

              precision    recall  f1-score   support

         0.0       0.17      0.20      0.18         5
         1.0       0.33      0.40      0.36         5
         2.0       1.00      0.50      0.67         4

    accuracy                           0.36        14
   macro avg       0.50      0.37      0.40        14
weighted avg       0.46      0.36      0.39        14

              precision    recall  f1-score   support

         0.0       0.67      0.80      0.73         5
         1.0       0.75      0.60      0.67         5
         2.0       0.50      0.50      0.50         4

    accuracy                           0.64        14
   macro avg       0.64      0.63      0.63        14
weighted avg       0.65      0.64      0.64        14

              precision    recall  f1-score   support

         0.0       0.50      0.60      0.55         5
         1.0       0.67      0.40      0.50         5
         2.0       0.60      0.75      0.67         4

    accuracy        

# uji test data

In [7]:
from sklearn.metrics import classification_report

# melakukan uji data
model.fit(X_train, y_train )
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.48      0.57      0.52        98
         1.0       0.59      0.37      0.46        97
         2.0       0.62      0.75      0.68        80

    accuracy                           0.55       275
   macro avg       0.56      0.56      0.55       275
weighted avg       0.56      0.55      0.54       275



In [13]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
param_grid={'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1,1],'kernel':['linear','poly']}
svc=svm.SVC(probability=True)
model=GridSearchCV(svc,param_grid)

In [ ]:
import matplotlib.pyplot as plt
url=input('Enter URL of Image :')
img=cv2.imread(url)
plt.imshow(img)
plt.show()

#Hasil Uji Klasifikasi pada dataset
Categories=['anjing','kucing','ikan']
img_resize=resize(img,(150,150,3))
l=[img_resize.flatten()]
model.fit(X_train,y_train)


# Mengecek hasil klasifikasi pada salah satu dataset
probability=model.predict_proba(l)
print(probability)
for ind,val in enumerate(Categories):
    print(f'{val} = {probability[0][ind]*100}%')
print("The predicted image is : "+str(model.predict(l)))
